# Extracting data from the Barcelona City Hall open data website
______

Here we are going to extract data from the [Barcelona City Hall Open Data Service](https://opendata-ajuntament.barcelona.cat/data/ca/dataset/divter/resource/ed515bb8-502b-4dff-96dc-769f72767919#additional-info) (this data is open and accessible to any user). Specifically, we are going to collect data on **socioeconomic indicators of the territorial division of the city of Barcelona**.

This information will be stored in a dataframe. 

As we will see, the data obtained are URLs that we will later use to download pdf files.

$~$


libraries we will use:

In [1]:
import requests
import pandas as pd
import numpy as np
import re
import os
import pprint
import pdfplumber

from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')



____

## Scraping the city hall website

It is true that the city council's website allows us open access to the data, but we could automate this.

<img src="./images/web_image2.png" width="500" height="340"/>


$~$

as we can see in the image above, the first thing we would have to do is download one or one of the csv files where we will find a series of URLs that we will then use to download the pdf files.

$~$


In [2]:
#url we want to scrape
url = "https://opendata-ajuntament.barcelona.cat/data/ca/dataset/divter"

#request the url 
web = requests.get(url)
#init soup 
soup = BeautifulSoup(web.content,"html.parser")



In [3]:
# find all the links in the page (div class="resource-actions-small">)
links = soup.find_all("div", class_="resource-actions-small")


links = [link.find("a") for link in links]

# get the links
links = [link.get("href") for link in links]


in the links list we have each one of the links that will allow us to download the csv files. let's create a dataframe with this information

In [4]:
#create a dataframe with the links
links = pd.DataFrame(links, columns=["link"])

In [5]:
#save the dataframe in a csv file as a backup 
links.to_csv("links.csv", index=False)

In [6]:
# read links.csv
links = pd.read_csv("links.csv")

In [7]:
links.head()

,link
0,https://opendata-ajuntament.barcelona.cat/data...
1,https://opendata-ajuntament.barcelona.cat/data...
2,https://opendata-ajuntament.barcelona.cat/data...
3,https://opendata-ajuntament.barcelona.cat/data...
4,https://opendata-ajuntament.barcelona.cat/data...


_____

## Download and save files with a url

We can use the dataframe "links" with the urls to download each of the .csv files. Note that each of the files refers to a specific year. The next cell will create a folder "csv_folder" where we will save the .cvs files.


In [8]:
#access the links, save the csv files in a folder called "csv_files"
#create a folder called "csv_files" if it doesn't exist
if not os.path.exists("csv_files"):
      os.mkdir("csv_files")

#loop through the links
for link in links["link"]:
      file_name = link.split("/")[-1]
      file_name = file_name.split("?")[0]
      file_name = file_name.split(".")[0]
      file_name = file_name + ".csv"
      print(file_name)
      with open(os.path.join("csv_files", file_name), "wb") as f:
         f.write(requests.get(link).content)



    

2019_divter.csv
2018_divter.csv
2017_divter.csv
2016_divter.csv
2015_divter.csv
2014_divter.csv
2013_divter.csv
2012_divter.csv
2011_divter.csv
2010_divter.csv
2009_divter.csv
2008_divter.csv


We open one of the files we have downloaded to copy the columns

In [236]:
#open one of the csv files 
df = pd.read_csv("csv_files/2019_divter.csv")

#check the columns names
print("the columns names are: ", df.columns)

the columns names are:  Index(['CODI_DIVISIO_TERRITORIAL', 'NOM_DIVISIO_TERRITORIAL',
       'CATEGORIA_DIVISIO', 'CODI_DIVISIO_TERRITORIAL_PARE',
       'URL_FITXA_DIVISIO_TERRITORIAL'],
      dtype='object')



let's create a new column any that will be used to know what year the information is from 

In [237]:
#create a list with the columns names and add the column "Any"
columns = list(df.columns)
columns.append("Any")

#create a new dataframe with the columns names
df_new = pd.DataFrame(columns=columns)

#put the column "any" in the first position
df_new = df_new[["Any"] + df_new.columns[:-1].tolist()]


In [238]:
df_new

,Any,CODI_DIVISIO_TERRITORIAL,NOM_DIVISIO_TERRITORIAL,CATEGORIA_DIVISIO,CODI_DIVISIO_TERRITORIAL_PARE,URL_FITXA_DIVISIO_TERRITORIAL


this empty dataframe will be used to fill it with the information that we are going to obtain from each of the csv files. 

In [241]:
#loop through the csv files
for file in os.listdir("csv_files"):
      #read the csv file
      df = pd.read_csv(os.path.join("csv_files", file))
      #get the year from the file name ""
      year = file.split("_")[0]
      #add the year to the dataframe in the column "Any"
      df["Any"] = year
      #concatenate the dataframes
      df_new = pd.concat([df_new, df], axis=0)
      

In [242]:
#show the first 5 rows of the dataframe
df_new.head()

,Any,CODI_DIVISIO_TERRITORIAL,NOM_DIVISIO_TERRITORIAL,CATEGORIA_DIVISIO,CODI_DIVISIO_TERRITORIAL_PARE,URL_FITXA_DIVISIO_TERRITORIAL
0,2017,B-01,el Raval,Barri,D-01,http://www.bcn.cat/estadistica/catala/dades/in...
1,2017,B-02,el Barri Gòtic,Barri,D-01,http://www.bcn.cat/estadistica/catala/dades/in...
2,2017,B-03,la Barceloneta,Barri,D-01,http://www.bcn.cat/estadistica/catala/dades/in...
3,2017,B-04,"Sant Pere, Santa Caterina i la Ribera",Barri,D-01,http://www.bcn.cat/estadistica/catala/dades/in...
4,2017,B-05,el Fort Pienc,Barri,D-02,http://www.bcn.cat/estadistica/catala/dades/in...


Now we have a dataframe with the columns:
+ Any = is the year of information
+ CODI_DIVISIO_TERRITORIAL: is the code of the territorial division managed by the city council of Barcelona for each of the neighborhoods.
+ NOM_DIVISIO_TERRITORIAL = this is the name of each of the barcelona neighborhoods.
+ CATEGORY_DIVISION = here there can be either (Barri) Neighborhood or District (Districte), 
+ CODI_DIVISIO_TERRITORIAL_PARE = this code is internal to the City Council of Barcelona.

+ URL_FITXA_DIVISIO_TERRITORIAL = this is the url from where we can download the pdf with the specific information of the neighborhood and the year.

Knowing this, we are only interested in the rows where we have url address, the following cells are useful for this purpose

In [13]:
#show if there are null values in the URL_FITXA_DIVISIO_TERRITORIAL
print("there are null values in the URL_FITXA_DIVISIO_TERRITORIAL: ", df_new["URL_FITXA_DIVISIO_TERRITORIAL"].isnull().values.any())
print("the number of null values in the URL_FITXA_DIVISIO_TERRITORIAL: ", df_new["URL_FITXA_DIVISIO_TERRITORIAL"].isnull().sum())

#show if there are empty values in the URL_FITXA_DIVISIO_TERRITORIAL
print("there are empty values in the URL_FITXA_DIVISIO_TERRITORIAL: ", df_new["URL_FITXA_DIVISIO_TERRITORIAL"].eq("").values.any())
print("the number of empty values in the URL_FITXA_DIVISIO_TERRITORIAL: ", df_new["URL_FITXA_DIVISIO_TERRITORIAL"].eq("").sum())



there are null values in the URL_FITXA_DIVISIO_TERRITORIAL:  True
the number of null values in the URL_FITXA_DIVISIO_TERRITORIAL:  120
there are empty values in the URL_FITXA_DIVISIO_TERRITORIAL:  False
the number of empty values in the URL_FITXA_DIVISIO_TERRITORIAL:  0


In [14]:
#drop the rows with null values in the URL_FITXA_DIVISIO_TERRITORIAL
df_new = df_new.dropna(subset=["URL_FITXA_DIVISIO_TERRITORIAL"])


In [15]:
#save the dataframe in a csv file
df_new.to_csv("project_data_barcelona.csv", index=False)


In [16]:
#read the csv file
df_new = pd.read_csv("project_data_barcelona.csv")


_____

## Downloading pdf files

Before we have obtained the URLs that will be used to download the .pdf files, now we are going to download them and save them in a folder that we are going to create and call pdf_files.

$~$

One of the problems with the city council's website is that the URLs they offer are corrupted. We have previously noticed this. The correct URLs are the ones we have called "url_base". The first thing we are going to do is to change the corrupted URLs for the correct ones.

In [17]:
# we previous know that the proper url to access the pdf files is the url in the column "URL_FITXA_DIVISIO_TERRITORIAL" have the following structure:

url_base = "https://ajuntament.barcelona.cat/estadistica/catala/Estadistiques_per_territori/Barris/Fitxes"

#we also previous know that the wrong url to access the pdf files is the url in the column "URL_FITXA_DIVISIO_TERRITORIAL" have the following structure:
url_wrong ="http://www.bcn.cat/estadistica/catala/dades/inf/barris"

#find the values in the column "URL_FITXA_DIVISIO_TERRITORIAL" that have the url_base
df_new["URL_FITXA_DIVISIO_TERRITORIAL"].str.contains(url_base).value_counts()




False    803
True      73
Name: URL_FITXA_DIVISIO_TERRITORIAL, dtype: int64

In [18]:
# for the values that have the url_wrong, we replace the url_wrong with the url_base
df_new.loc[df_new["URL_FITXA_DIVISIO_TERRITORIAL"].str.contains(url_wrong), "URL_FITXA_DIVISIO_TERRITORIAL"] = df_new.loc[df_new["URL_FITXA_DIVISIO_TERRITORIAL"].str.contains(
    url_wrong), "URL_FITXA_DIVISIO_TERRITORIAL"].str.replace(url_wrong, url_base)


In [19]:
#save the dataframe in a csv file for backup.
df_new.to_csv("project_data_barcelona_url_clean.csv", index=False)


In [20]:
#read the csv file
df = pd.read_csv("project_data_barcelona_url_clean.csv")

_____

$~$

Now, with the correct URl we can download the information. The next cell is used to create a folder "pdf_files", and download the files and save them in that folder. In addition we are going to call each of the files by the year to which the information refers and the name of the neighborhood, that is to say, it will have the following structure:

*year_neighborhood name.pdf*

In [21]:
#create a folder to save the pdf files called "pdf_files"
#check if the folder exists and if not create it
import time
import urllib.request
errors = []
if not os.path.exists("pdf_files"):
    os.mkdir("pdf_files")

#loop through the dataframe and download the pdf files
for i in range(len(df)):
    #get the url of the pdf file
    url = df["URL_FITXA_DIVISIO_TERRITORIAL"][i]
    #get the name of the file Any+NOM_DIVISIO+.pdf
    file_name = str(df["Any"][i]) + "_" + df["NOM_DIVISIO_TERRITORIAL"][i] + ".pdf"
    #create the path to save the file
    path = os.path.join("pdf_files", file_name)
    #try to download the file in files and save them in the folder pdf_files if the download fails print the error, the url and the file name. save the error with the name of the file in a list
    try:
        urllib.request.urlretrieve(url, path)
        #print the name of the file downloaded

    except Exception as e:
        print(e)
        print(url)
        print(file_name)
        errors.append(file_name)
    #wait 1 second to avoid the error 429
    # time.sleep(1)


2017_el Raval.pdf
2017_el Barri Gòtic.pdf
2017_la Barceloneta.pdf
2017_Sant Pere, Santa Caterina i la Ribera.pdf
2017_el Fort Pienc.pdf
2017_la Sagrada Família.pdf
2017_la Dreta de l'Eixample.pdf
2017_l'Antiga Esquerra de l'Eixample.pdf
2017_la Nova Esquerra de l'Eixample.pdf
2017_Sant Antoni.pdf
2017_el Poble-sec.pdf
2017_la Marina del Prat Vermell.pdf
2017_la Marina de Port.pdf
2017_la Font de la Guatlla.pdf
2017_Hostafrancs.pdf
2017_la Bordeta.pdf
2017_Sants - Badal.pdf
2017_Sants.pdf
2017_les Corts.pdf
2017_la Maternitat i Sant Ramon.pdf
2017_Pedralbes.pdf
2017_Vallvidrera, el Tibidabo i les Planes.pdf
2017_Sarrià.pdf
2017_les Tres Torres.pdf
2017_Sant Gervasi - la Bonanova.pdf
2017_Sant Gervasi - Galvany.pdf
2017_el Putxet i el Farró.pdf
2017_Vallcarca i els Penitents.pdf
2017_el Coll.pdf
2017_la Salut.pdf
2017_la Vila de Gràcia.pdf
2017_el Camp d'en Grassot i Gràcia Nova.pdf
2017_el Baix Guinardó.pdf
2017_Can Baró.pdf
2017_el Guinardó.pdf
2017_la Font d'en Fargues.pdf
2017_el Car


We keep a list with the names of the files we have downloaded and also a list with the names of the files we have not been able to download.

In [197]:
#save the name of the files in a list with the path. here are the files that we already have downloaded
files = [os.path.join("pdf_files", file) for file in os.listdir("pdf_files")]

#print the number of files downloaded
print("Number of files downloaded: {}".format(len(files)))


Number of files downloaded: 803


In [198]:
#print the number of files with errors
print("Number of files with errors: {}".format(len(errors)))

_________

## Extracting information from pdf files

we already have the pdf files now what we are going to do is to extract the information from the files. 

To do this, the first thing we are going to do is to define a dictionary with the following keys

+ Barri = is the name of the neighborhood
+ Any = the year the information refers to
+ Població = is the population of the neighborhood for that year.
+ Superfície_(km2) = is the area in km2 of the neighborhood
+ Densitat_(hab/km2) = is the density of inhabitants per square kilometer. 
+ Dones = Total number of women living in the neighborhood.
+ Homes = Total number of men living in the neighborhood.
+ edat_0_14 = percentage of the total population of the neighborhood aged 0 to 14 years old
+ edat_15_24 = percentage of the total population of the neighborhood aged 15 to 24 years old
+ edat_25_64 = percentage of the total population of the neighborhood aged 25 to 64 years old
+ edat_65_i_mes = percentage of the total population of the neighborhood over the age of 65
+ naixement_Barcelona = population of the neighborhood (in percentage) born in Barcelona
+ population of the neighborhood (in percentage) born outside of Barcelona but in the region of catalonia 
+ naixement_Resta_Espanya = population of the neighborhood (in percentage) born in 
other parts of spain
+ naixement_Estranger = foreign-born population of the neighborhood (in percentage)
+ nacionalitat_Espanyols = percentage of the neighborhood's population with Spanish nationality
+ nacionalitat_Estrangers = percentage of the neighborhood's population with foreign nationality
+ Titulats_superiors = Percentage of the population with higher education in the neighborhood
+ Taxa_natalitat= neighborhood birth rate per 1000 inhabitants
+ mes_de_65_anys_viu_sola = percentage of the neighborhood population over 65 years old living alone
+ Index_de_sobreenvelliment =  index population >75 years/ population >65 years*100
+ Turismes = tourists in the neighborhood (persons) per 1,000 inhabitants
+ Motos = motorcycles (individuals) / 1,000 inhab.
+ Ciclomotors = Cyclomotors (physical persons) / 1,000 inhab.
+ aturats_registrats = Number of registered unemployed
+ Renda_familiar = Family income available per inhabitant


In [200]:
# create a empty dataframe with the columns we want.
# the columns are: "Any","Barri","Població","Superfície_(km2)",
# "Densitat_(hab/km2)","per_sexe_Dones","per_sexe_Homes","per_edat_(en %)_0-14",
# "per_edat_(en %)_15-24", "per_edat_(en %)_25-64", "per_edat_(en %)_65 i mes"
# "lloc de naixement_(en %)_Barcelona",""lloc de naixement_(en %)_Resta Catalunya"
#"lloc de naixement_(en %)_Resta Espanya", "lloc de naixement_(en %)_Estranger"
# per_nacionalitat_(en %)_Espanyols", "per_nacionalitat_(en %)_Estrangers"
#"%_Titulats superiors", "Taxa natalitat(1.000 hab)","més_de_65_anys_que_viu_sola_(%)"
#"Índex_de_sobreenvelliment", "Turismes_(persones físiques)_1.000 hab"
#"Motos_(persones_físiques)1.000_hab", "Ciclomotors_(persones físiques)1.000_hab"
#"Nombre_d'aturats_registrats", "Renda_familiar_disponible_per_habitant_(2017)"


columns = ["Barri","Any","Població", "Superfície_(km2)", "Densitat_(hab/km2)", "Dones", "Homes", "edat_0_14",
           "edat_15_24", "edat_25_64", "edat_65_i_mes", "naixement_Barcelona",
           "naixement_Resta_Catalunya", "naixement_Resta_Espanya", "naixement_Estranger",
           "nacionalitat_Espanyols", "nacionalitat_Estrangers", "Titulats_superiors",
           "Taxa_natalitat", "mes_de_65_anys_viu_sola", "Index_de_sobreenvelliment",
           "Turismes", "Motos",
           "Ciclomotors", "aturats_registrats", "Renda_familiar"]



In [201]:
#create a dictionary with the columns as keys and the values as empty lists
data_structure = {}
for column in columns:
    data_structure[column] = None



Now we are going to create a series of regular expressions that will allow us to extract the information from the pdf files.

In [204]:
# create a regex to get the data from the pdf files.

#patter to find the year(any) it is a  interger number after the string "SOCIOECONÒMICS "
any = re.compile(r"SOCIOECONÒMICS\s+(\d+)")

# pattert to find the poblation it is a number with 1 or more digits and a comma or a dot and 1 or more digits after the comma or dot after the word "Població"
poblacio = re.compile(r'Població\s+(\d+[\.,]\d+|\d+)')

# pattert to find a number with 1 or more digits after the word "Superfície (km2) "
superficie = re.compile(r'Superfície\s\(km2\)\s+(\d+[\.,]\d+|\d+)')


# pattert to find a number with 1 or more digits after the word "Densitat (hab/km2) "
densitat = re.compile(r'Densitat\s\(hab/km2\)\s+(\d+[\.,]\d+|\d+)')

# pattert to find a number with 1 or more digits after the word "Dones" or a interger after the word "Dones"
dones = re.compile(r'Dones\s+(\d+[\.,]\d+|\d+)')

# pattert to find a number with 1 or more digits after the word "Homes", or a interger after the word "Homes"
homes = re.compile(r'Homes\s+(\d+[\.,]\d+|\d+)')

# pattert to find a number with 1 or more digits after 0-14
edat_0_14 = re.compile(r'[0].[1][4]\s+(\d+[\.,]\d+|\d+)')



# pattert to find a number with 1 or more digits after 15"-"24, or a interger after 15"-"24" "
edat_15_24 = re.compile(r'[1][5].[2][4]\s+(\d+[\.,]\d+|\d+)')


# pattert to find a number with 1 or more digits after the word "25-64"
edat_25_64 = re.compile(r'[2][5].[6][4]\s+(\d+[\.,]\d+|\d+)')


# pattert to find a number with 1 or more digits after the word "65 i mes " o 65 i mes"
edat_65_i_mes = re.compile(r'[6][5]\s[i]\s[m][e][s]\s+(\d+[\.,]\d+|\d+)')


# pattert to find a number with 1 or more digits after the word "Barcelona"
naixement_Barcelona = re.compile(r'Barcelona\s+(\d+[\.,]\d+|\d+)')

# pattert to find a number with 1 or more digits after the word "Resta Catalunya"
naixement_Resta_Catalunya = re.compile(r'Resta\sCatalunya\s+(\d+[\.,]\d+|\d+)')

# pattert to find a number with 1 or more digits after the word "Resta Espanya"
naixement_Resta_Espanya = re.compile(r'ta\sEspanya\s+(\d+[\.,]\d+|\d+)')

# pattert to find a number with 1 or more digits after the word "Estranger"
naixement_Estranger = re.compile(r'Estranger\s+(\d+[\.,]\d+|\d+)')

# pattert to find a number with 1 or more digits after the word "Espanyols"
nacionalitat_Espanyols = re.compile(r'Espanyols\s+(\d+[\.,]\d+|\d+)')

# pattert to find a number with 1 or more digits after the word "Estrangers"
nacionalitat_Estrangers = re.compile(r'Estrangers\s+(\d+[\.,]\d+|\d+)')

# pattert to find a number with 1 or more digits after the word "CFGS (1)"
Titulats_superiors = re.compile(r'CFGS\s\(1\)\s+(\d+[\.,]\d+|\d+)')

# pattert to find a number with 1 or more digits after the word "Taxa natalitat / 1.000 hab."
Taxa_natalitat = re.compile(r'Taxa natalitat / 1.000 hab.\s+(\d+[\.,]\d+|\d+)')

# pattert to find a number with 1 or more digits after the word "Població de més de 65 anys que viu sola (%) (2)"
mes_de_65_anys_viu_sola = re.compile(r'Població de més de 65 anys que viu sola \(%\) \(2\)\s+(\d+[\.,]\d+|\d+)')

# pattert to find a number with 1 or more digits after the word Index de sobreenvelliment (?)"
Index_de_sobreenvelliment = re.compile(r"Índex de sobreenvelliment\s+\(\d+\)\s+(\d+[\.,]\d+|\d+)")

# pattert to find a number with 1 or more digits after the word "Turismes (persones físiques) / 1.000 hab (4)"
Turismes = re.compile(r"Turismes \(persones físiques\)\s\/\s1\.000\shab\s\(\d\)\s+(\d+[\.,]\d+|\d+)")

# pattert to find a number with 1 or more digits after the word "Motos (persones físiques) / 1.000 hab (4)"
Motos = re.compile(r"(Motos\s\(persones\sfísiques\)\s\/\s1\.000\shab\s\(\d\)\s|Motos\s\(persones\sfísiques\)\s\/\s1\.000\shab\s)+(\d+[\.,]\d+|\d+)")

# pattert to find a number with 1 or more digits after the word "Ciclomotors (persones físiques) / 1.000 hab (4)"

Ciclomotors = re.compile(r'Ciclomotors \(persones físiques\)\s\/\s1\.000\shab\s\(\d\)\s+(\d+[\.,]\d+|\d+)')

# pattert to find any interger after the word "(5)"
aturats_registrats = re.compile(r"Nombre d'aturats registrats\s\(\d\)\s+(\d+[\.,]\d+|\d+)")

# pattert to find a number with 1 or more digits after the word "(2017)"
Renda_familiar = re.compile(r"Renda\sfamiliar\sdisponible\sper\shabitant\s\(\d{4}\)\s+(\d+[\.,]\d+|\d+)")


once the regular expressions have been defined, we will extract the information from the pdfs and save what we are interested in 

In [220]:
# here we are going to create a list of all the patterns. This will be useful later
# also we are going to use pdfplumber to extract the text from the pdf
values = []


for file in files:


    data = data_structure
    data["Barri"] = file[:-4][15:]
    # print(file[:-4][15:])

    with pdfplumber.open(file) as pdf:
        # check if the pdf has 2 pages
        if len(pdf.pages) == 2:
        
            page = pdf.pages[1]
            text = page.extract_text()
            for i in text.split('\n'):
                
                if any.search(i):
                    # finding the "Any" pattern that matches the text and adding it to the dictionary
                    data['Any'] = any.search(i).group(1)
                    # print(any.search(i).group(1))
                    continue

                    # find the "Població" pattern that matches the text and adding it to the dictionary
                if poblacio.search(i):
                    data["Població"] = poblacio.search(i).group(1)
                    # print(poblacio.search(i).group(1))
                
                    continue

                    # find the "superfície" pattern that matches the text and adding it to the dictionary
                if superficie.search(i):
                
                    data['Superfície_(km2)'] = superficie.search(i).group(1)
                    # print(superficie.search(i).group(1))

                    continue

                    # find the "densitat" pattern that matches the text and adding it to the dictionary
                if densitat.search(i):
                    
                    data["Densitat_(hab/km2)"] = densitat.search(i).group(1)
                    # print(densitat.search(i).group(1))

                    continue

                    # find the Dones pattern that matches the text and adding it to the dictionary
                if dones.search(i):
                    
                    data["Dones"]= dones.search(i).group(1)
                    # print(dones.search(i).group(1))

                    continue

                    # find the Homes pattern that matches the text and adding it to the dictionary
                if homes.search(i):
                        # print(homes.search(i).group(1))
                        data["Homes"] = homes.search(i).group(1)
                        continue

                    # find the edat_0_14 pattern that matches the text and adding it to the dictionary
                if edat_0_14.search(i):
                        # print(edat_0_14.search(i).group(1))
                        data["edat_0_14"] = edat_0_14.search(i).group(1)
                        continue

                # find the edat_15_64 pattern that matches the text and adding it to the dictionary
                if edat_15_24.search(i):
                        # print(edat_15_24.search(i).group(1))
                        data["edat_15_24"] = edat_15_24.search(i).group(1)
                        continue
                        #find the edat_25_64 pattern that matches the text and adding it to the dictionary
                if edat_25_64.search(i):
                        # print(edat_25_64.search(i).group(1))
                        data["edat_25_64"] = edat_25_64.search(i).group(1)
                        continue

                    # find the pattern  edat_65_i_mes that matches the text and adding it to the dictionary
                if edat_65_i_mes.search(i):
                        # print(edat_65_i_mes.search(i).group(1))
                        data ["edat_65_i_mes"]= edat_65_i_mes.search(i).group(1)
                        continue


                    # find the naixement_Barcelona pattern that matches the text and adding it to the dictionary
                if naixement_Barcelona.search(i):
                        # print(naixement_Barcelona.search(i).group(1))
                        data["naixement_Barcelona"] = naixement_Barcelona.search(i).group(1)
                        continue

                    # find the naixement_Resta_Catalunya pattern that matches the text and adding it to the dictionary
                if naixement_Resta_Catalunya.search(i):
                        # print(naixement_Resta_Catalunya.search(i).group(1))
                        data["naixement_Resta_Catalunya"]= naixement_Resta_Catalunya.search(i).group(1)
                        continue

                    # find the naixement_Resta_Espanya pattern that matches the text and adding it to the dictionary
                if naixement_Resta_Espanya.search(i):
                        # print(naixement_Resta_Espanya.search(i).group(1))
                        data["naixement_Resta_Espanya"]= naixement_Resta_Espanya.search(i).group(1)
                        continue

                    # find the naixement_Extranger pattern that matches the text and adding it to the dictionary
                if naixement_Estranger.search(i):
                        # print(naixement_Estranger.search(i).group(1))
                        data["naixement_Estranger"] = naixement_Estranger.search(i).group(1)
                        continue

                    # find the nacionalitat_Espanyols pattern that matches the text and adding it to the dictionary
                if nacionalitat_Espanyols.search(i):
                        # print(nacionalitat_Espanyols.search(i).group(1))
                        data["nacionalitat_Espanyols"] = nacionalitat_Espanyols.search(i).group(1)
                        continue

                    # find the nacionalitat_Estrangers

                if nacionalitat_Estrangers.search(i):
                        # print(nacionalitat_Estrangers.search(i).group(1))
                        data["nacionalitat_Estrangers"] = nacionalitat_Estrangers.search(i).group(1)
                        continue

                    # find the Titulats_superiors pattern that matches the text and adding it to the dictionary
                if Titulats_superiors.search(i):
                        # print(Titulats_superiors.search(i).group(1))
                        data["Titulats_superiors"] = Titulats_superiors.search(i).group(1)
                        continue

                    # find the Taxa_natalitat pattern that matches the text and adding it to the dictionary
                if Taxa_natalitat.search(i):
                        # print(Taxa_natalitat.search(i).group(1))
                        data["Taxa_natalitat"] = Taxa_natalitat.search(i).group(1)
                        continue

                    # find the mes_de_65_anys_viu_sola pattern that matches the text and adding it to the dictionary
                if mes_de_65_anys_viu_sola.search(i):
                        # print(mes_de_65_anys_viu_sola.search(i).group(1))
                        data["mes_de_65_anys_viu_sola"] = mes_de_65_anys_viu_sola.search(i).group(1)
                        continue

                    # find the Index_de_sobreenvelliment and get the number
                if Index_de_sobreenvelliment.search(i):
                        # print(Index_de_sobreenvelliment.search(i).group(1))
                        data["Index_de_sobreenvelliment"] = Index_de_sobreenvelliment.search(i).group(1)
                        continue

                    # find the Turisme pattern that matches the text and adding it to the dictionary
                if Turismes.search(i):
                        # print(Turismes.search(i).group(1))
                        data["Turismes"] = Turismes.search(i).group(1)
                        continue

                    # find the Motos pattern that matches the text and adding it to the dictionary

                if Motos.search(i):
                    # print(Motos.search(i).group(1))
                    data["Motos"] = Motos.search(i).group(2)
                    continue

                    #find the Ciclomotors pattern that matches the text and adding it to the dictionary

                if Ciclomotors.search(i):
                    # print(Ciclomotors.search(i).group(1))
                    data["Ciclomotors"] = Ciclomotors.search(i).group(1)
                    continue
                
                
                    # find the aturats_registrats pattern that matches the text and adding it to the dictionary
                if aturats_registrats.search(i):
                    # print(aturats_registrats.search(i).group(1))
                    data["aturats_registrats"] = aturats_registrats.search(i).group(1)
                    continue   

                    #find the renta familiar pattern that matches the text and adding it to the dictionary
                if Renda_familiar.search(i):
                    # print(Renda_familiar.search(i).group(1))
                    data["Renda_familiar"] = Renda_familiar.search(i).group(1)
                    continue
        else:
            print(file + " this .pdf file no have the same pages as the others")     
    values.append(data.copy())
#    

pdf_files/2015_el Putxet i el Farró.pdf this .pdf file no have the same pages as the others


we create a dataframe with the extracted information

In [261]:
data_barris = pd.DataFrame(values)
data_barris

,Barri,Any,Població,Superfície_(km2),Densitat_(hab/km2),Dones,Homes,edat_0_14,edat_15_24,edat_25_64,...,nacionalitat_Estrangers,Titulats_superiors,Taxa_natalitat,mes_de_65_anys_viu_sola,Index_de_sobreenvelliment,Turismes,Motos,Ciclomotors,aturats_registrats,Renda_familiar
0,"Sant Pere, Santa Caterina i la Ribera",2012,22.873,"1,1",20.527,11.479,11.394,"9,7","8,4","66,0",...,"38,5","33,7","9,3","30,4","54,1","237,8","97,9","24,5",613,"89,1"
1,el Clot,2012,27.201,"0,7",39.085,14.079,13.122,"13,1","9,2","60,4",...,"14,4","21,6","9,3","30,4","54,1","237,8","95,5","24,5",613,"81,1"
2,el Putxet i el Farró,2016,27.201,"0,8",34.996,16.176,13.441,"14,3","10,0","54,7",...,"13,0","48,9","9,3","30,4","54,1","237,8","213,7","24,5",613,"141,0"
3,la Trinitat Nova,2016,27.201,"0,6",12.982,3.813,3.458,"14,6","10,2","56,5",...,"18,7","7,5","9,3","30,4","54,1","237,8","62,5","24,5",613,"35,6"
4,Torre Baró,2018,2.931,"1,7",1.653,1.457,1.474,"17,9","12,5","56,7",...,"18,7","8,2","9,7","21,8","46,2","290,7","77,8","24,5",169,"46,5"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798,la Vila de Gràcia,2013,52.586,"1,3",39.657,28.233,24.353,"10,2","7,5","62,7",...,"17,6","10,3","8,5","19,6","45,9","379,9","128,5","28,1",93,"146,6"
799,el Poblenou,2014,52.586,"1,5",21.634,17.182,16.243,"15,9","7,2","60,0",...,"16,6","29,3","8,5","19,6","45,9","379,9","94,2","28,1",93,"89,6"
800,Vilapicina i la Torre Llobeta,2013,25.527,"0,6",45.021,13.798,11.729,"11,9","8,6","55,2",...,"12,2","18,7","8,5","19,6","45,9","379,9","88,5","28,1",93,"71,7"
801,la Barceloneta,2017,25.527,"1,3",11.335,7.489,7.404,"8,2","8,9","65,0",...,"36,0","28,6","8,5","19,6","45,9","379,9","110,3","28,1",93,"84,8"


In [262]:
#show the info of the dataframe
data_barris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 803 entries, 0 to 802
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Barri                      803 non-null    object
 1   Any                        803 non-null    object
 2   Població                   803 non-null    object
 3   Superfície_(km2)           803 non-null    object
 4   Densitat_(hab/km2)         803 non-null    object
 5   Dones                      803 non-null    object
 6   Homes                      803 non-null    object
 7   edat_0_14                  803 non-null    object
 8   edat_15_24                 803 non-null    object
 9   edat_25_64                 803 non-null    object
 10  edat_65_i_mes              803 non-null    object
 11  naixement_Barcelona        803 non-null    object
 12  naixement_Resta_Catalunya  803 non-null    object
 13  naixement_Resta_Espanya    803 non-null    object
 14  naixement_

now we are going to clean up the data

In [263]:
#replace "," with "." in the dataframe
data_barris = data_barris.replace(',','.', regex=True)


In [267]:
data_barris

,Barri,Any,Població,Superfície_(km2),Densitat_(hab/km2),Dones,Homes,edat_0_14,edat_15_24,edat_25_64,...,nacionalitat_Estrangers,Titulats_superiors,Taxa_natalitat,mes_de_65_anys_viu_sola,Index_de_sobreenvelliment,Turismes,Motos,Ciclomotors,aturats_registrats,Renda_familiar
0,Sant Pere. Santa Caterina i la Ribera,2012,22.873,1.1,20.527,11.479,11.394,9.7,8.4,66.0,...,38.5,33.7,9.3,30.4,54.1,237.8,97.9,24.5,613,89.1
1,el Clot,2012,27.201,0.7,39.085,14.079,13.122,13.1,9.2,60.4,...,14.4,21.6,9.3,30.4,54.1,237.8,95.5,24.5,613,81.1
2,el Putxet i el Farró,2016,27.201,0.8,34.996,16.176,13.441,14.3,10.0,54.7,...,13.0,48.9,9.3,30.4,54.1,237.8,213.7,24.5,613,141.0
3,la Trinitat Nova,2016,27.201,0.6,12.982,3.813,3.458,14.6,10.2,56.5,...,18.7,7.5,9.3,30.4,54.1,237.8,62.5,24.5,613,35.6
4,Torre Baró,2018,2.931,1.7,1.653,1.457,1.474,17.9,12.5,56.7,...,18.7,8.2,9.7,21.8,46.2,290.7,77.8,24.5,169,46.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798,la Vila de Gràcia,2013,52.586,1.3,39.657,28.233,24.353,10.2,7.5,62.7,...,17.6,10.3,8.5,19.6,45.9,379.9,128.5,28.1,93,146.6
799,el Poblenou,2014,52.586,1.5,21.634,17.182,16.243,15.9,7.2,60.0,...,16.6,29.3,8.5,19.6,45.9,379.9,94.2,28.1,93,89.6
800,Vilapicina i la Torre Llobeta,2013,25.527,0.6,45.021,13.798,11.729,11.9,8.6,55.2,...,12.2,18.7,8.5,19.6,45.9,379.9,88.5,28.1,93,71.7
801,la Barceloneta,2017,25.527,1.3,11.335,7.489,7.404,8.2,8.9,65.0,...,36.0,28.6,8.5,19.6,45.9,379.9,110.3,28.1,93,84.8


In [273]:
#change the type of the columns to int 
to_change = ["Any"]
data_barris[to_change] = data_barris[to_change].astype(int)

#change Població: extract "." and transform to int
data_barris["Població"] = data_barris["Població"].str.replace(".","")
data_barris["Població"] = data_barris["Població"].astype(int)


In [274]:
# safe the dataframe to a csv file as a backup
data_barris.to_csv("data_barris.csv", index=False)


In [275]:
#read the csv file
data_barris = pd.read_csv("data_barris.csv")